In [6]:
%config IPCompleter.greedy=True

import numpy as np
import os
import cv2 as cv
import tensorflow as tf
from tensorflow import keras

In [7]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=90,        # Randomly rotate images by up to 30 degrees
    width_shift_range=0.2,    # Randomly shift images horizontally by 20%
    height_shift_range=0.2,   # Randomly shift images vertically by 20%
    shear_range=0.2,          # Shear images
    zoom_range=0.2,           # Zoom into images
    horizontal_flip=True,     # Flip images horizontally
    brightness_range=[0.8, 1.2],  # Randomly adjust brightness
    fill_mode='nearest'       # Fill empty pixels with nearest valid value
)

In [8]:
__dirname = r"train"

img_arr = []
feature = []

for img in os.listdir(__dirname):
    cur_img = cv.imread(os.path.join(__dirname,img))
    cur_img = cv.resize(cur_img,(224,224))
    cur_img = np.expand_dims(cur_img, axis=0)

    # Apply augmentation
    augmented_images = datagen.flow(cur_img, batch_size=1)
    
    for _ in range(1):
        augmented_img = next(augmented_images)[0].astype(np.uint8)
        img_arr.append(augmented_img)
        feature.append(int(img[-6:-5]))

In [9]:
img_arr = np.array(img_arr)
feature = np.array(feature)

In [10]:
img_arr.shape,feature.shape

((18000, 224, 224, 3), (18000,))

In [ ]:
index = 17

cv.imshow("some",img_arr[index])
cv.waitKey(0)

In [ ]:
cv.destroyAllWindows()


In [11]:
index = 65

def convert(index):
    img = img_arr[index]
    # Apply thresholding
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    thresh_img = cv.threshold(img, 110, 255, cv.THRESH_BINARY)[1]
    return thresh_img


In [12]:
new_img_arr = []
for i in range(0,len(img_arr)):
    new_img_arr.append(convert(i))

In [13]:
print(len(new_img_arr))

18000


In [14]:
img_arr = new_img_arr
img_arr = np.array(img_arr)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
img_arr,x_val,feature,y_val = train_test_split(img_arr,feature,train_size=0.7)

In [17]:
inp_shp = img_arr.shape[1:]+(1,)

model = keras.Sequential()
model.add(keras.layers.Conv2D(128,(3,3), activation="relu",input_shape=inp_shp))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(64,(3,3),activation="relu"))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(32,(3,3),activation="relu"))
model.add(keras.layers.MaxPooling2D((2,2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.Dense(64, activation="relu"))
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dense(np.unique(feature).shape[0], activation="softmax"))

C:\Users\Harsh\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
model.compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics=['accuracy']
)

In [ ]:
model.fit(img_arr, feature, epochs=10, validation_data = (x_val,y_val))

Epoch 1/10
394/394 ━━━━━━━━━━━━━━━━━━━━ 937s 2s/step - accuracy: 0.3734 - loss: 1.7939 - val_accuracy: 0.6144 - val_loss: 0.8866
Epoch 2/10
298/394 ━━━━━━━━━━━━━━━━━━━━ 3:27 2s/step - accuracy: 0.7385 - loss: 0.6628

In [ ]:
model.save("finger_2.keras")